In [4]:
import h5py
import numpy as np
import pandas as pd

# Loading

In [5]:
def load_exps_and_labels(detections):
    all_exps, all_labels = [], []

    counter = 0

    for exp_name, data in detections.groupby('Exp'):
        counter += 1

        try:
            file = h5py.File(f'../../data/real/{exp_name}.h5', 'r')
        except:
            print(f'Couldnt load exp {exp_name}')
            continue

        exp = []
        
        for channel_id in range(1, 17):
            channel = file[f'Chn{channel_id}']
            exp.append(channel)
        
        exp = np.swapaxes(exp, 1, 0)

        chirps_ids = []

        for _, (frame_id, depth) in data[['Frame', 'Depth']].iterrows():
            chirps_ids = chirps_ids + list(np.linspace(frame_id*127, frame_id*128+127, 128, dtype=np.uint16))
            all_labels = all_labels + list(np.repeat(depth, 128))

        exp = exp[chirps_ids]

        all_exps = all_exps + list(exp)
    
    all_exps = np.array(all_exps)
    all_labels = np.array([all_labels]).T
    all_exps[:, :, :50] = 0
    
    return all_exps.astype(np.float32), all_labels.astype(np.float32)
        

In [6]:
FILE_NAME = 'EXP_17_M'

In [7]:
all_detections = pd.read_csv('../../data/real/TABULAR_CSV_ALL_CLASSES.csv')

In [8]:
detections = all_detections[all_detections['Exp'].str.contains('EXP_17_M')].query('Class == "Motorcycle" and Depth <= 25')

In [9]:
pd.unique(detections['Exp'])

array(['EXP_17_M'], dtype=object)

In [10]:
raw_data, labels = load_exps_and_labels(detections)

In [11]:
labels.shape

(6144, 1)

In [12]:
np.save(f'../../data/preprocessed/{FILE_NAME}_chirps.npy', raw_data)

In [13]:
raw_data = np.swapaxes(raw_data, 2, 1)

In [14]:
raw_data.shape

(6144, 1024, 16)

# Scale

## Data Scale

In [15]:
print(f'Mean: {raw_data.mean()}, Std: {raw_data.std()}')

Mean: 183.6147003173828, Std: 1001.0029907226562


In [16]:
data_min = raw_data.min()
data_max = raw_data.max()

In [17]:
print(f'Min: {data_min}, Max: {data_max}')

Min: -3884.0, Max: 4772.0


In [18]:
a, b = -1, 1

In [19]:
np.subtract(raw_data, data_min, out=raw_data)
np.multiply(raw_data, (b - a), out=raw_data)
np.divide(raw_data, (data_max - data_min), out=raw_data)
raw_data = np.add(raw_data, a, out=raw_data)

In [20]:
normalized_mean = raw_data.mean()

In [21]:
print(f'Mean: {normalized_mean}')

Mean: -0.060162827372550964


### Label Generator Scale

In [22]:
labels_mean = labels.mean()
labels_std = labels.std()

In [23]:
print(f'Mean: {labels_mean}, Std: {labels_std}')

Mean: 10.981254577636719, Std: 7.1911773681640625


In [24]:
labels_generator = (labels - labels_mean) / labels_std

### Label Discriminator Scale

In [25]:
labels_min = labels.min()
labels_max = labels.max()

In [26]:
labels_discriminator = labels - labels_mean
labels_discriminator = a + ((labels_discriminator - labels_min)*(b-a)) / (labels_max - labels_min)

# Save

In [27]:
np.save(f'../../data/preprocessed/{FILE_NAME}_chirps_scaled.npy', raw_data)
np.save(f'../../data/preprocessed/{FILE_NAME}_chirps_labels.npy', (labels_generator, labels_discriminator))